In [2]:
#import libraries
import pandas as pd
import numpy as np

In [3]:
#upload csv
fw17_18=pd.read_csv('fall17_winter18.csv')


In [4]:
#extract necessary columns
f17_w=fw17_18.loc[:, ["StudentID", "Subject", "StartScore", "EndScore", "StudentGrade"]]


In [5]:
#rename columns
f17w = f17_w.rename(columns={"StartScore":"Fall (2017)", "EndScore":"Winter (2018)"})
f17w.head()

,StudentID,Subject,Fall (2017),Winter (2018),StudentGrade
0,20,Language Usage,228.0,233.0,8
1,23,Language Usage,231.0,230.0,4
2,30,Language Usage,NaN,209.0,3
3,31,Language Usage,NaN,223.0,8
4,001-0001,Language Usage,189.0,197.0,2


In [6]:
#upload csv
ws_18=pd.read_csv('Winter18_Spring18.csv')

In [8]:
#extract necessary columns
s_18=ws_18.loc[:, ["StudentID", "Subject", "EndScore", "StudentGrade"]]

In [25]:
#rename columns
s18=s_18.rename(columns={"EndScore":"Spring (2018)"})
s18.head()

,StudentID,Subject,Spring (2018),StudentGrade
0,001-0002,Language Usage,229,5
1,001-0001,Language Usage,201,2
2,001-0004,Language Usage,244,5
3,7,Language Usage,210,3
4,001-0008,Language Usage,227,5


In [26]:
#upload csv
fw18_19=pd.read_csv('Fall18_Winter19.csv')

In [27]:
#extract necessary columns
fw18_19=fw18_19.loc[:, ["StudentID", "Subject", "StartScore", "EndScore", "StudentGrade"]]

In [28]:
#rename columns
fw1819 = fw18_19.rename(columns={"StartScore":"Fall (2018)", "EndScore":"Winter (2019)"})
fw1819.head()

,StudentID,Subject,Fall (2018),Winter (2019),StudentGrade
0,001-0073,Language Usage,NaN,204.0,5
1,001-0002,Language Usage,228.0,NaN,***
2,001-0001,Language Usage,202.0,NaN,***
3,001-0003,Language Usage,189.0,201.0,2
4,001-0004,Language Usage,246.0,250.0,6


In [29]:
#upload csv
ws_19=pd.read_csv('Winter19_Spring19.csv')

In [30]:
#extract necessary columns
ws_19=ws_19.loc[:, ["StudentID", "Subject", "EndScore", "StudentGrade"]]

In [31]:
#rename columns
ws19 = ws_19.rename(columns={"EndScore":"Spring (2019)"})
ws19.head()

,StudentID,Subject,Spring (2019),StudentGrade
0,101,Language Usage,232.0,6
1,102,Language Usage,225.0,7
2,001-0073,Language Usage,214.0,5
3,001-0002,Language Usage,232.0,6
4,001-0005,Language Usage,219.0,5


In [32]:
#combine tables
merge_table = pd.merge(f17w, s18, on=["StudentID", "Subject"], how="outer")

In [33]:
#combine tables and rename columns
merge_tables2 = pd.merge(merge_table, fw1819, on=["StudentID", "Subject"], how="outer")
merge_table2=merge_tables2.rename(columns={"StudentGrade_x":"grade1", "StudentGrade_y":"grade2"})

In [34]:
#combine tables and replace *** with null values
merge_table3 = pd.merge(merge_table2, ws19, on=["StudentID", "Subject"], how="outer")
merge_table4=merge_table3.replace({'***':np.NaN})

In [35]:
#combine grade columns from each spreadsheet to get rid of null values 
merge_table4['Grade']=merge_table4['StudentGrade_x'].fillna(merge_table4['StudentGrade_y'].fillna(merge_table4['grade1'].fillna(merge_table4['grade2'])))
merge_table4.head()

,StudentID,Subject,Fall (2017),Winter (2018),grade1,Spring (2018),grade2,Fall (2018),Winter (2019),StudentGrade_x,Spring (2019),StudentGrade_y,Grade
0,20,Language Usage,228.0,233.0,8,232.0,8,NaN,NaN,NaN,NaN,NaN,8
1,23,Language Usage,231.0,230.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,30,Language Usage,NaN,209.0,3,210.0,3,NaN,NaN,NaN,NaN,NaN,3
3,31,Language Usage,NaN,223.0,8,225.0,8,NaN,NaN,NaN,NaN,NaN,8
4,001-0001,Language Usage,189.0,197.0,2,201.0,2,202.0,NaN,NaN,NaN,NaN,2


In [36]:
#upload csv and isolate student id column
cons=pd.read_csv('Consolidated_Test_Data.csv')
final_id=cons.loc[:, ['Student']]


In [37]:
#rename column to prepare for merge
con=final_id.rename(columns={"Student":"StudentID"})


In [38]:
#combine tables to include only the Student IDs on the final form
merge_table5 = pd.merge(merge_table4, con, on=["StudentID"], how="inner")
merge_table5['Grade']=merge_table5['Grade'].replace('K', 0)
merge_table5.head()

,StudentID,Subject,Fall (2017),Winter (2018),grade1,Spring (2018),grade2,Fall (2018),Winter (2019),StudentGrade_x,Spring (2019),StudentGrade_y,Grade
0,001-0001,Language Usage,189.0,197.0,2,201.0,2,202.0,NaN,NaN,NaN,NaN,2
1,001-0001,Mathematics,195.0,208.0,2,211.0,2,211.0,NaN,NaN,222.0,3,3
2,001-0001,Reading,189.0,197.0,2,204.0,2,208.0,NaN,NaN,208.0,3,3
3,001-0001,Science - General Science,194.0,186.0,2,196.0,2,196.0,NaN,NaN,NaN,NaN,2
4,001-0002,Language Usage,224.0,234.0,5,229.0,5,228.0,NaN,NaN,232.0,6,6


In [95]:
#restructure data set into a pivot table
final=merge_table5.pivot_table(index=['StudentID', 'Grade'], columns='Subject', values=['Fall (2017)', 'Winter (2018)', 'Spring (2018)', 'Fall (2018)', 'Winter (2019)', 'Spring (2019)'])
final.to_csv('consolidated_test_grades.csv')
final

Fall (2017)                                                \
Subject         Language Usage Mathematics Reading Science - General Science   
StudentID Grade                                                                
001-0001  2              189.0         NaN     NaN                     194.0   
          3                NaN       195.0   189.0                       NaN   
001-0002  6              224.0       225.0   229.0                     210.0   
001-0003  2              174.0       203.0   191.0                       NaN   
001-0004  6              235.0       277.0   239.0                     229.0   
001-0005  5                NaN         NaN     NaN                       NaN   
001-0006  5                NaN         NaN     NaN                       NaN   
001-0008  6              219.0       237.0   236.0                     213.0   
001-0009  8              232.0       251.0   244.0                     228.0   
001-0010  1                NaN         NaN     NaN                       NaN   
001-0011  0                NaN         NaN     NaN                       NaN   
001-0012  4                NaN         NaN     NaN                       NaN   
001-0013  7                NaN         NaN     NaN                       NaN   
001-0014  5                NaN         NaN     NaN                       NaN   
001-0015  5                NaN         NaN     NaN                       NaN   
001-0018  6                NaN         NaN     NaN                       NaN   
001-0019  5              224.0       232.0   219.0                     211.0   
001-0020  5              204.0       202.0   199.0                     198.0   
001-0021  1                NaN         NaN     NaN                       NaN   
001-0022  3                NaN         NaN     NaN                       NaN   
001-0023  6                NaN         NaN     NaN                       NaN   
001-0024  3              167.0       229.0   188.0                       NaN   
001-0025  5              208.0       215.0   215.0                     206.0   
001-0026  1                NaN         NaN     NaN                       NaN   
001-0027  6                NaN         NaN     NaN                       NaN   
001-0028  5                NaN         NaN     NaN                       NaN   
001-0029  7                NaN         NaN     NaN                       NaN   
001-0030  5              214.0       213.0   208.0                     204.0   
001-0031  7              236.0       242.0   241.0                     224.0   
001-0032  5                NaN         NaN     NaN                       NaN   
...                        ...         ...     ...                       ...   
001-0049  3                NaN         NaN     NaN                       NaN   
001-0050  6                NaN         NaN     NaN                       NaN   
001-0051  0                NaN         NaN     NaN                       NaN   
001-0052  1                NaN         NaN     NaN                       NaN   
001-0053  3                NaN         NaN     NaN                       NaN   
001-0054  1                NaN         NaN     NaN                       NaN   
001-0055  4                NaN         NaN     NaN                       NaN   
001-0056  6                NaN         NaN     NaN                       NaN   
001-0057  2                NaN         NaN     NaN                       NaN   
001-0058  5                NaN         NaN     NaN                       NaN   
001-0059  6              180.0       182.0   182.0                       NaN   
001-0060  4                NaN         NaN     NaN                       NaN   
001-0062  8                NaN         NaN     NaN                       NaN   
001-0063  5              199.0       212.0   200.0                     197.0   
001-0065  6                NaN         NaN     NaN                       NaN   
001-0066  4                NaN         NaN     NaN                       NaN   
001-0068  1        